### Classifier, Dataset 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import *
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split as tts
lgbr = LGBMRegressor()
lgb = LGBMClassifier()

In [4]:
import pandas as pd
df = pd.read_pickle('/content/drive/Shareddrives/빅콘테스트/최종데이터/log_rank_final_ver1')
df.columns

Index(['user_id', 'insert_time', 'loan_limit', 'loan_rate', 'is_applied',
       'credit_score', 'yearly_income', 'desired_amount', 'existing_loan_cnt',
       'existing_loan_amt', 'kospi', 'kospi_change', 'age', 'working_period',
       'missing_cnt', 'personal_rehabilitation', 'excess_of_existing_loan',
       'max_insert_ratio_time', 'credit_grade', 'credit_card', 'seomin_fin',
       'day', 'prod_loan_limit', 'prod_rate_avg', 'dlrate', 'gender',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_OTHERINCOME',
       'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_기타', 'employment_type_일용직',
       'employment_type_정규직', 'houseown_type_기타가족소유', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_BUSINESS',
       'purpose_BUYCAR', 'purpose_BUYHOUSE', 'purpose_ETC',
       'purpose_HOUSEDEPOSIT', 'purpose_INVEST', 'purpose_LIVING',
       'pur

In [5]:
min_kos_ch = min(df['kospi_change'])
max_kos_ch = max(df['kospi_change'])
df['kospi_change'] = df['kospi_change'].apply(lambda x : ((x - min_kos_ch) / max_kos_ch))
df.head().T

,0,1,2,3,4
user_id,346970.0,545882.0,558819.0,867912.0,867912.0
insert_time,2022-05-09 08:44:57,2022-05-09 10:54:51,2022-05-09 10:41:05,2022-05-09 07:14:15,2022-05-09 07:16:40
loan_limit,0.330551,0.269792,0.269792,0.41802,0.41802
loan_rate,0.810811,0.810811,0.702703,0.810811,0.810811
is_applied,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
loan_limit_rank,1.0,2.0,4.0,22.0,13.0
loan_rate_rank,1.0,1.0,3.0,19.0,25.0
already_bank,0,0,0,0,0
already_product,0,0,0,0,0


- 모든 dummies column이 0이라면 새로운 식별자가 된다.
- dummy 유형 별 한 개의 column은 불필요하다고 봄.
- 기타 항목에 해당하는 column들을 drop하고 학습 진행

In [6]:
object_lst = ['loan_limit_rank', 'loan_rate_rank', 'already_bank', 'already_product']
df[object_lst] = df[object_lst].astype('float')
df = df.dropna()

features = ['loan_limit', 'loan_rate', 'credit_score', 'yearly_income', 'desired_amount', 'existing_loan_cnt',
       'existing_loan_amt', 'kospi', 'kospi_change', 'age', 'working_period',
       'missing_cnt', 'personal_rehabilitation', 'excess_of_existing_loan',
       'max_insert_ratio_time', 'credit_grade', 'credit_card', 'seomin_fin',
       'day', 'prod_loan_limit', 'prod_rate_avg', 'dlrate', 'gender',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_일용직', 'employment_type_정규직', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_BUSINESS',
       'purpose_BUYCAR', 'purpose_BUYHOUSE', 'purpose_HOUSEDEPOSIT', 'purpose_INVEST', 'purpose_LIVING',
       'purpose_SWITCHLOAN', 'k_means_cluster_0', 'k_means_cluster_1',
       'k_means_cluster_2', 'Android', 'iOS', 'iPadOS', 'update_activeness', 'CompleteIDCertification', 'EndLoanApply',
       'GetCreditInfo', 'Login', 'OpenApp', 'SignUp', 'StartLoanApply',
       'UseDSRCalc', 'UseLoanManage', 'UsePrepayCalc', 'ViewLoanApplyIntro',
       'many', 'only_open', 'only_start', 'ended', 'extra', 'loan_limit_rank',
       'loan_rate_rank', 'already_bank', 'already_product']

deleted_features = ['employment_type_기타', 'income_type_OTHERINCOME','houseown_type_기타가족소유', 
                    'purpose_ETC', 'k_means_cluster_3', 'index']
label = ['is_applied']

x = df[features]
y = df[label]

### not use Sampling method

In [7]:
x_train, x_test, y_train, y_test = tts(x, y, random_state=5, test_size=0.2)

In [8]:
lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=1000,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, random_state=1798,
               reg_alpha=0.01, reg_lambda=0.1, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
train_pred = lgb.predict(x_train)
test_pred = lgb.predict(x_test)
all_pred = lgb.predict(x)

print('============ Sampling하지 않은 model Score ============')
print('--------------- 훈련 Data Set에서 Score ---------------')
print('훈련 dataset에 대한 AUC Score       :', roc_auc_score(y_train, train_pred))
print('훈련 dataset에 대한 F1 Score        :', f1_score(y_train, train_pred))
print('훈련 testset에 대한 Precision Score :', precision_score(y_train, train_pred))
print('훈련 dataset에 대한 Recall Score    :', recall_score(y_train, train_pred))

print('--------------- 검증 Data Set에서 Score ---------------')
print('검증 dataset에 대한 AUC Score       :', roc_auc_score(y_test, test_pred))
print('검증 dataset에 대한 F1 Score        :', f1_score(y_test, test_pred))
print('검증 testset에 대한 Precision Score :', precision_score(y_test, test_pred))
print('검증 dataset에 대한 Recall Score    :', recall_score(y_test, test_pred))

print('--------------- 전체 Data Set에서 Score ---------------')
print('전체 dataset에 대한 AUC Score       :', roc_auc_score(y, all_pred))
print('전체 dataset에 대한 F1 Score        :', f1_score(y, all_pred))
print('전체 testset에 대한 Precision Score :', precision_score(y, all_pred))
print('전체 dataset에 대한 Recall Score    :', recall_score(y, all_pred))

============ Sampling하지 않은 model Score ============
--------------- 훈련 Data Set에서 Score ---------------
훈련 dataset에 대한 AUC Score       : 0.5701390218670365
훈련 dataset에 대한 F1 Score        : 0.2387145689956841
훈련 testset에 대한 Precision Score : 0.6930110055498072
훈련 dataset에 대한 Recall Score    : 0.14419136268803356
--------------- 검증 Data Set에서 Score ---------------
검증 dataset에 대한 AUC Score       : 0.5543433582757423
검증 dataset에 대한 F1 Score        : 0.19005198672901594
검증 testset에 대한 Precision Score : 0.5746899439134213
검증 dataset에 대한 Recall Score    : 0.11385154697256608
--------------- 전체 Data Set에서 Score ---------------
전체 dataset에 대한 AUC Score       : 0.5669787235982806
전체 dataset에 대한 F1 Score        : 0.22904259240990066
전체 testset에 대한 Precision Score : 0.6702525298568693
전체 dataset에 대한 Recall Score    : 0.1381210625853237


In [9]:
matrix = confusion_matrix(y, all_pred)
matrix_df = pd.DataFrame(matrix, columns=['N', 'P'], index=['N', 'P'])
matrix_df

,N,P
N,5190595,21702
P,275260,44112


## 1:1 Sampling // f1: 0.30

### UnderSampling 1:1

1대 1 비율로 random undersampling을 진행했다..

In [10]:
undersample = RandomUnderSampler(sampling_strategy='majority')
x_under, y_under = undersample.fit_resample(x, y)

X_train = pd.concat([x_under, y_under], axis=1)
print(len(X_train[X_train['is_applied']==1]), len(X_train[X_train['is_applied']==0]))

319372 319372


In [11]:
x_train, x_test, y_train, y_test = tts(x_under, y_under, random_state=5, test_size=0.2)

lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=1000,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, random_state=1798,
               reg_alpha=0.01, reg_lambda=0.1, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
train_pred = lgb.predict(x_train)
test_pred = lgb.predict(x_test)
all_pred = lgb.predict(x)

print('============ Sampling하지 않은 model Score ============')
print('--------------- 훈련 Data Set에서 Score ---------------')
print('훈련 dataset에 대한 AUC Score       :', roc_auc_score(y_train, train_pred))
print('훈련 dataset에 대한 F1 Score        :', f1_score(y_train, train_pred))
print('훈련 testset에 대한 Precision Score :', precision_score(y_train, train_pred))
print('훈련 dataset에 대한 Recall Score    :', recall_score(y_train, train_pred))

print('--------------- 검증 Data Set에서 Score ---------------')
print('검증 dataset에 대한 AUC Score       :', roc_auc_score(y_test, test_pred))
print('검증 dataset에 대한 F1 Score        :', f1_score(y_test, test_pred))
print('검증 testset에 대한 Precision Score :', precision_score(y_test, test_pred))
print('검증 dataset에 대한 Recall Score    :', recall_score(y_test, test_pred))

print('--------------- 전체 Data Set에서 Score ---------------')
print('전체 dataset에 대한 AUC Score       :', roc_auc_score(y, all_pred))
print('전체 dataset에 대한 F1 Score        :', f1_score(y, all_pred))
print('전체 testset에 대한 Precision Score :', precision_score(y, all_pred))
print('전체 dataset에 대한 Recall Score    :', recall_score(y, all_pred))

============ Sampling하지 않은 model Score ============
--------------- 훈련 Data Set에서 Score ---------------
훈련 dataset에 대한 AUC Score       : 0.858862009510928
훈련 dataset에 대한 F1 Score        : 0.8635025057913304
훈련 testset에 대한 Precision Score : 0.8359484931988743
훈련 dataset에 대한 Recall Score    : 0.8929348677000156
--------------- 검증 Data Set에서 Score ---------------
검증 dataset에 대한 AUC Score       : 0.8172563977209921
검증 dataset에 대한 F1 Score        : 0.8239092390320439
검증 testset에 대한 Precision Score : 0.795212301240608
검증 dataset에 대한 Recall Score    : 0.85475489889188
--------------- 전체 Data Set에서 Score ---------------
전체 dataset에 대한 AUC Score       : 0.8347678286479584
전체 dataset에 대한 F1 Score        : 0.3274864164584687
전체 testset에 대한 Precision Score : 0.20090169467438804
전체 dataset에 대한 Recall Score    : 0.8852967699109502


In [12]:
matrix = confusion_matrix(y, all_pred)
matrix_df = pd.DataFrame(matrix, columns=['N', 'P'], index=['N', 'P'])
matrix_df

,N,P
N,4087686,1124611
P,36633,282739


## Sampling 실험 비율

### UnderSampling 1:3 // f1: 0.436

In [13]:
undersample = RandomUnderSampler(sampling_strategy=0.33)
x_under, y_under = undersample.fit_resample(x, y)

X_train = pd.concat([x_under, y_under], axis=1)
print(len(X_train[X_train['is_applied']==1]), len(X_train[X_train['is_applied']==0]))

319372 967793


In [14]:
x_train, x_test, y_train, y_test = tts(x_under, y_under, random_state=5, test_size=0.2)

lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=1000,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, random_state=1798,
               reg_alpha=0.01, reg_lambda=0.1, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
train_pred = lgb.predict(x_train)
test_pred = lgb.predict(x_test)
all_pred = lgb.predict(x)

print('============ Sampling하지 않은 model Score ============')
print('--------------- 훈련 Data Set에서 Score ---------------')
print('훈련 dataset에 대한 AUC Score       :', roc_auc_score(y_train, train_pred))
print('훈련 dataset에 대한 F1 Score        :', f1_score(y_train, train_pred))
print('훈련 testset에 대한 Precision Score :', precision_score(y_train, train_pred))
print('훈련 dataset에 대한 Recall Score    :', recall_score(y_train, train_pred))

print('--------------- 검증 Data Set에서 Score ---------------')
print('검증 dataset에 대한 AUC Score       :', roc_auc_score(y_test, test_pred))
print('검증 dataset에 대한 F1 Score        :', f1_score(y_test, test_pred))
print('검증 testset에 대한 Precision Score :', precision_score(y_test, test_pred))
print('검증 dataset에 대한 Recall Score    :', recall_score(y_test, test_pred))

print('--------------- 전체 Data Set에서 Score ---------------')
print('전체 dataset에 대한 AUC Score       :', roc_auc_score(y, all_pred))
print('전체 dataset에 대한 F1 Score        :', f1_score(y, all_pred))
print('전체 testset에 대한 Precision Score :', precision_score(y, all_pred))
print('전체 dataset에 대한 Recall Score    :', recall_score(y, all_pred))

============ Sampling하지 않은 model Score ============
--------------- 훈련 Data Set에서 Score ---------------
훈련 dataset에 대한 AUC Score       : 0.7934316869694298
훈련 dataset에 대한 F1 Score        : 0.7034236765562597
훈련 testset에 대한 Precision Score : 0.7590355977360511
훈련 dataset에 대한 Recall Score    : 0.655404452108481
--------------- 검증 Data Set에서 Score ---------------
검증 dataset에 대한 AUC Score       : 0.7613261349060879
검증 dataset에 대한 F1 Score        : 0.6533712880459499
검증 testset에 대한 Precision Score : 0.7114037339984595
검증 dataset에 대한 Recall Score    : 0.6040926929545879
--------------- 전체 Data Set에서 Score ---------------
전체 dataset에 대한 AUC Score       : 0.7824131584428164
전체 dataset에 대한 F1 Score        : 0.43660873416675317
전체 testset에 대한 Precision Score : 0.32996940916443773
전체 dataset에 대한 Recall Score    : 0.645087859925103


In [15]:
matrix = confusion_matrix(y, all_pred)
matrix_df = pd.DataFrame(matrix, columns=['N', 'P'], index=['N', 'P'])
matrix_df

,N,P
N,4793950,418347
P,113349,206023


### UnderSampling 1:5 // f1: 0.446
- sampling 비율이 커질 수록 더 많은 1을 놓친다.
- 대신 0을 1로 예측하는 비율은 줄어든다.
- 0을 더 잘 예측할 수 있게 된다.

In [16]:
undersample = RandomUnderSampler(sampling_strategy=0.2)
x_under, y_under = undersample.fit_resample(x, y)

X_train = pd.concat([x_under, y_under], axis=1)
print(len(X_train[X_train['is_applied']==1]), len(X_train[X_train['is_applied']==0]))

319372 1596860


In [17]:
x_train, x_test, y_train, y_test = tts(x_under, y_under, random_state=5, test_size=0.2)

lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=1000,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, random_state=1798,
               reg_alpha=0.01, reg_lambda=0.1, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
train_pred = lgb.predict(x_train)
test_pred = lgb.predict(x_test)
all_pred = lgb.predict(x)

print('============ Sampling하지 않은 model Score ============')
print('--------------- 훈련 Data Set에서 Score ---------------')
print('훈련 dataset에 대한 AUC Score       :', roc_auc_score(y_train, train_pred))
print('훈련 dataset에 대한 F1 Score        :', f1_score(y_train, train_pred))
print('훈련 testset에 대한 Precision Score :', precision_score(y_train, train_pred))
print('훈련 dataset에 대한 Recall Score    :', recall_score(y_train, train_pred))

print('--------------- 검증 Data Set에서 Score ---------------')
print('검증 dataset에 대한 AUC Score       :', roc_auc_score(y_test, test_pred))
print('검증 dataset에 대한 F1 Score        :', f1_score(y_test, test_pred))
print('검증 testset에 대한 Precision Score :', precision_score(y_test, test_pred))
print('검증 dataset에 대한 Recall Score    :', recall_score(y_test, test_pred))

print('--------------- 전체 Data Set에서 Score ---------------')
print('전체 dataset에 대한 AUC Score       :', roc_auc_score(y, all_pred))
print('전체 dataset에 대한 F1 Score        :', f1_score(y, all_pred))
print('전체 testset에 대한 Precision Score :', precision_score(y, all_pred))
print('전체 dataset에 대한 Recall Score    :', recall_score(y, all_pred))

============ Sampling하지 않은 model Score ============
--------------- 훈련 Data Set에서 Score ---------------
훈련 dataset에 대한 AUC Score       : 0.7307835479043651
훈련 dataset에 대한 F1 Score        : 0.5924062309998531
훈련 testset에 대한 Precision Score : 0.7303011331981689
훈련 dataset에 대한 Recall Score    : 0.4983148092991174
--------------- 검증 Data Set에서 Score ---------------
검증 dataset에 대한 AUC Score       : 0.7022811601089366
검증 dataset에 대한 F1 Score        : 0.5376456333302264
검증 testset에 대한 Precision Score : 0.6694675270414558
검증 dataset에 대한 Recall Score    : 0.44919636182407174
--------------- 전체 Data Set에서 Score ---------------
전체 dataset에 대한 AUC Score       : 0.7228786437798081
전체 dataset에 대한 F1 Score        : 0.44707669379800524
전체 testset에 대한 Precision Score : 0.4121722312058087
전체 dataset에 대한 Recall Score    : 0.48843981313327406


In [18]:
matrix = confusion_matrix(y, all_pred)
matrix_df = pd.DataFrame(matrix, columns=['N', 'P'], index=['N', 'P'])
matrix_df

,N,P
N,4989823,222474
P,163378,155994


### UnderSampling 1:7 // f1_score 0.412

In [19]:
undersample = RandomUnderSampler(sampling_strategy=(1/7))
x_under, y_under = undersample.fit_resample(x, y)

X_train = pd.concat([x_under, y_under], axis=1)
print(len(X_train[X_train['is_applied']==1]), len(X_train[X_train['is_applied']==0]))

319372 2235604


In [20]:
x_train, x_test, y_train, y_test = tts(x_under, y_under, random_state=5, test_size=0.2)

lgb = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.2, max_depth=1000,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, random_state=1798,
               reg_alpha=0.01, reg_lambda=0.1, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
train_pred = lgb.predict(x_train)
test_pred = lgb.predict(x_test)
all_pred = lgb.predict(x)

print('============ Sampling하지 않은 model Score ============')
print('--------------- 훈련 Data Set에서 Score ---------------')
print('훈련 dataset에 대한 AUC Score       :', roc_auc_score(y_train, train_pred))
print('훈련 dataset에 대한 F1 Score        :', f1_score(y_train, train_pred))
print('훈련 testset에 대한 Precision Score :', precision_score(y_train, train_pred))
print('훈련 dataset에 대한 Recall Score    :', recall_score(y_train, train_pred))

print('--------------- 검증 Data Set에서 Score ---------------')
print('검증 dataset에 대한 AUC Score       :', roc_auc_score(y_test, test_pred))
print('검증 dataset에 대한 F1 Score        :', f1_score(y_test, test_pred))
print('검증 testset에 대한 Precision Score :', precision_score(y_test, test_pred))
print('검증 dataset에 대한 Recall Score    :', recall_score(y_test, test_pred))

print('--------------- 전체 Data Set에서 Score ---------------')
print('전체 dataset에 대한 AUC Score       :', roc_auc_score(y, all_pred))
print('전체 dataset에 대한 F1 Score        :', f1_score(y, all_pred))
print('전체 testset에 대한 Precision Score :', precision_score(y, all_pred))
print('전체 dataset에 대한 Recall Score    :', recall_score(y, all_pred))

============ Sampling하지 않은 model Score ============
--------------- 훈련 Data Set에서 Score ---------------
훈련 dataset에 대한 AUC Score       : 0.6806220863468958
훈련 dataset에 대한 F1 Score        : 0.49900879961423555
훈련 testset에 대한 Precision Score : 0.7161122135895314
훈련 dataset에 대한 Recall Score    : 0.38291932275596346
--------------- 검증 Data Set에서 Score ---------------
검증 dataset에 대한 AUC Score       : 0.6542509244797978
검증 dataset에 대한 F1 Score        : 0.4401713782005289
검증 testset에 대한 Precision Score : 0.6394663172315206
검증 dataset에 대한 Recall Score    : 0.3355838959739935
--------------- 전체 Data Set에서 Score ---------------
전체 dataset에 대한 AUC Score       : 0.6741795706113407
전체 dataset에 대한 F1 Score        : 0.41895511508839106
전체 testset에 대한 Precision Score : 0.4771115280450289
전체 dataset에 대한 Recall Score    : 0.3734359931365304


In [21]:
matrix = confusion_matrix(y, all_pred)
matrix_df = pd.DataFrame(matrix, columns=['N', 'P'], index=['N', 'P'])
matrix_df

,N,P
N,5081589,130708
P,200107,119265
